<a href="https://colab.research.google.com/github/AfricArxiv/repo-dev/blob/master/osf_preprints_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import json


preprintsApiUrl = "https://api.osf.io/v2/providers/preprints/africarxiv/preprints/?page[size]=57"
preprintsRequest = requests.get( url = preprintsApiUrl )
preprintsRequestContent = json.loads( preprintsRequest.content )

keys_list = [ 'titl', 'date', 'description', 'link', 'doi', 'authors', 'subjects', 'tags' ]
values_list = []

preprintMeta = []
preprintsMeta = []

subjects_list = []
tags_list = []

author = []
preprintAuthors = []
authors = []


for preprint in preprintsRequestContent['data']:

  authorApiUrl = preprint['relationships']['contributors']['links']['related']['href']
  authorRequest = requests.get( url = authorApiUrl )
  authorRequestContent = json.loads( authorRequest.content )

  for preprintAuthor in authorRequestContent['data']:

    author.append( preprintAuthor['embeds']['users']['data']['attributes']['full_name'] )
    author.append( preprintAuthor['embeds']['users']['data']['links']['html'] )

    preprintAuthors.append( author )
    author = []

  for subject in preprint['attributes']['subjects']:
    for subj in subject:
      subjects_list.append( subj['text'] )

  for tag in preprint['attributes']['tags']:
    tags_list.append( tag )

  preprintMeta.append( preprint['attributes']['title'] )
  preprintMeta.append( preprint['attributes']['date_published'] )
  preprintMeta.append( preprint['attributes']['description'] )
  preprintMeta.append( preprint['links']['html'] )
  preprintMeta.append( preprint['links']['preprint_doi'] )

  preprintMeta.append( preprintAuthors )
  preprintAuthors = []

  preprintMeta.append( subjects_list )
  subjects_list = []

  preprintMeta.append( tags_list )
  tags_list = []
  
  preprintsMeta.append( preprintMeta )
  preprintMeta = []
    

In [0]:
preprintsMetaJson = json.dumps( [dict( zip( keys_list, row ) ) for row in preprintsMeta], indent=1 )